In [18]:
from src.service import nlp
import numpy as np

In [66]:
from pymilvus import *
client = connections.connect("default", host="localhost", port="19530")

In [ ]:
from pymilvus import *
client = connections.connect("default", host="localhost", port="19530")

In [ ]:
from pymilvus import *
client = connections.connect("default", host="localhost", port="19530")

In [65]:
client.list_collections()

AttributeError: 'NoneType' object has no attribute 'list_collections'

In [ ]:
COLLECTION_NAME = 'demo_films'
PARTITION_NAME = 'Movie'

pk = FieldSchema(name='pk', dtype=DataType.INT64, is_primary=True, auto_id=False)
field = FieldSchema(name='vec', dtype=DataType.FLOAT_VECTOR, dim=384)
schema = CollectionSchema(fields=[pk, field], description="movie recommender: demo films")

if utility._get_connection("default").has_collection(COLLECTION_NAME): # drop the same collection created before
    collection = Collection(COLLECTION_NAME)
    collection.drop()
else:
    collection = Collection(name=COLLECTION_NAME, schema=schema)
    partition = collection.create_partition(PARTITION_NAME)
    print("Collection & partition are successfully created.")
print(collection)    

In [ ]:
testStrings =["test string 1","test string 2","test sting3"]
ids2 = [1,2,3]
embeddings2 =[]
for string in testStrings:
    embedding = await nlp.embed(string)
    embeddings2.append(embedding.numpy())


In [ ]:
print(collection)

In [60]:
print(collection)
if collection.num_entities != 0:
    print(COLLECTION_NAME + " is not empty!")  
else:
    mr = collection.insert(data=[ids2,embeddings2], partition_name=PARTITION_NAME)
    print("here")

print("Record count in collection: " + str(collection.num_entities))

<Collection>:
-------------
<name>: demo_films
<partitions>: [{"name": "_default", "collection_name": "demo_films", "description": ""}, {"name": "Movie", "collection_name": "demo_films", "description": ""}]
<description>: movie recommender: demo films
<schema>: {
  auto_id: False
  description: movie recommender: demo films
  fields: [{
    name: pk
    description: 
    type: 5
    is_primary: True
    auto_id: False
  }, {
    name: vec
    description: 
    type: 101
    params: {'dim': 384}
  }]
}

here
Record count in collection: 3


In [262]:
from pymilvus import *
client = connections.connect("default", host="localhost", port="19530")

In [205]:
COLLECTION_NAME = 'Messages'
PARTITION_NAME = 'Reddit'
collection = Collection(COLLECTION_NAME)



In [115]:
index_param = {
    "metric_type": "L2",
    "index_type":"IVF_FLAT",
    "params":{"nlist":1024}
}
collection.create_index(field_name="vec", index_params=index_param)

Status(code=0, message='')

In [263]:
collection.load() # load collection memory before search

topK = 10
SEARCH_PARAM = {
    "metric_type":"L2",
    "params":{"nprobe": 20},
}

In [264]:
string="Email Automation"
embedding = await nlp.embed(string)

In [265]:
type(embedding.numpy())

numpy.ndarray

In [266]:
results = collection.search([embedding.numpy()],"vec",param=SEARCH_PARAM, limit=topK, expr=None, output_fields=None)

In [207]:
import os
import asyncpraw
import redis
from dotenv import load_dotenv
load_dotenv()

r = redis.Redis(
host=os.environ.get("REDIS_HOST"),
port=os.environ.get("REDIS_PORT"),
password=os.environ.get("REDIS_PASSWORD"))

reddit = asyncpraw.Reddit(
client_id="lhCAjKV2lNlVn0msO6isUQ",
client_secret="sDVFmyCNxZSrZ6_Tod19dvqcv3az6w",
user_agent="Ironman",
)
connections.connect("default", host="localhost", port="19530")




In [237]:
@staticmethod
def decode(i: int) -> str:
    b= i.to_bytes(((i.bit_length() + 7) // 8), byteorder='big')
    return b.decode()  

In [267]:
I = []
for x in results:
    for y in x.ids:
        I.append(decode(y))
I

['xeuf3v',
 'x7lgpu',
 'wv8012',
 'x1gkyb',
 'x7slzl',
 'ws0kk0',
 'wowva1',
 'wtacom',
 'wty1ro',
 'xeugm8']

In [268]:
recall_results = []
for x in I:
    recall_results.append(r.hget("message:{}".format(x),"text").decode('utf-8'))
recall_results

['What ESP or SMTP do you use for cold emails?',
 "How to get founder's emails at scale?",
 'Cold Email Template Ideas',
 'Affiliate software',
 'Content Marketing Tool',
 'Building an automatic tasks manager for Social Media events.',
 'MS-900 - Microsoft 365 Fundamentals Full Course to help those that need it',
 'ABM tool for EU',
 'How to find social profiles from email and phone?',
 'Any affordable tools/software to manage digital contract for an advertising agency?']

In [273]:
async def semanticSearch(query):
    print(query)
    topK = 10
    SEARCH_PARAM = {
        "metric_type":"L2",
        "params":{"nprobe": 10},
    }
    embedding = await nlp.embed(query)
    print(type(embedding.numpy()))
    results = collection.search([embedding.numpy()],"vec",param=SEARCH_PARAM, limit=topK, expr=None, output_fields=None)
    I = []
    for x in results:
        for y in x.ids:
            I.append(decode(y))
    recall_results = []
    for x in I:
        recall_results.append(r.hget("message:{}".format(x),"text").decode('utf-8'))
    return recall_results


In [274]:
results = await semanticSearch("Rate this idea")

Rate this idea
<class 'numpy.ndarray'>


In [272]:
results

['Rate our idea',
 'Suggestion',
 'Idea Feedback',
 'Looking For Feedback',
 'Idea validation - for people who love the outdoors!',
 'Idea validation...Second venture, hoping to start off with a blast.',
 'saas idea needed',
 '$10 for 30 minutes of my time',
 'Weekly Feedback Post - SaaS Products, Ideas, Companies',
 'Weekly Feedback Post - SaaS Products, Ideas, Companies']

In [1]:
import asyncpraw
reddit = asyncpraw.Reddit(
client_id="lhCAjKV2lNlVn0msO6isUQ",
client_secret="sDVFmyCNxZSrZ6_Tod19dvqcv3az6w",
user_agent="Ironman",
)

In [2]:
multireddit = await reddit.subreddit("SaaS",fetch=True)
multireddit.id

'2qkq6'

In [27]:
import os
from dotenv import load_dotenv
from supabase import create_client, Client


load_dotenv()  # take environment variables from .env.
url: str = os.environ.get("SUPABASE_URL")
key: str = os.environ.get("SUPABASE_KEY")
supabase: Client = create_client(url, key)

channels = supabase.table('Subreddit').select("*").execute()

In [28]:
channels.data

[{'id': 4, 'channel_name': 'r/SaaS', 'ext_id': '2qkq6'},
 {'id': 5, 'channel_name': 'r/Entrepreneur', 'ext_id': '2qldo'}]